In [1]:
spark

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1,application_1705787158430_0002,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
%%configure -f
{"conf": {
    "spark.sql.extensions": "io.delta.sql.DeltaSparkSessionExtension",
    "spark.sql.catalog.spark_catalog": "org.apache.spark.sql.delta.catalog.DeltaCatalog"
    }
}        

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
2,application_1705787158430_0003,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
2,application_1705787158430_0003,pyspark,idle,Link,Link,None,✔


In [4]:
df_json = spark.read.json("s3://pod-academy-ingestion/sales/cgrrmsv_20240120220414.json")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
df_json.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+-------+----------+--------+--------+------+--------------+
|         city|     id|price_unit| product|quantity|seller|     timestamp|
+-------------+-------+----------+--------+--------+------+--------------+
|San Francisco|cgrrmsv|        20|Lt3p55iF|      19|   Eva|20240120220414|
+-------------+-------+----------+--------+--------+------+--------------+

In [6]:
schema = df_json.schema
print(schema)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

StructType([StructField('city', StringType(), True), StructField('id', StringType(), True), StructField('price_unit', LongType(), True), StructField('product', StringType(), True), StructField('quantity', LongType(), True), StructField('seller', StringType(), True), StructField('timestamp', StringType(), True)])

In [8]:
df = spark.readStream \
    .schema(schema) \
    .format("json") \
    .option("header", "true") \
    .option("maxFilesPerTrigger", "1") \
    .load("s3://pod-academy-ingestion/sales/*.json")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
df.isStreaming

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

True

In [10]:
df.createOrReplaceTempView("df")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
df_01 = spark.sql("""
    select
        a.id, 
        a.seller,
        a.city,
        a.price_unit, 
        a.product, 
        a.quantity,  
        round(a.price_unit * a.quantity,2) as total_price,
        a.timestamp
        
    from 
        df a
""")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
df_01.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- id: string (nullable = true)
 |-- seller: string (nullable = true)
 |-- city: string (nullable = true)
 |-- price_unit: long (nullable = true)
 |-- product: string (nullable = true)
 |-- quantity: long (nullable = true)
 |-- total_price: long (nullable = true)
 |-- timestamp: string (nullable = true)

In [14]:
df_01.writeStream \
    .format("delta") \
    .outputMode("append") \
    .option("checkpointLocation", "s3://pod-academy-ingestion/checkpoint") \
    .start("s3://pod-academy-processed/sales")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…